# CityBikes

In [1]:
import requests
import pandas as pd
from pprint import pprint

In [78]:
city_bike_base_url = 'http://api.citybik.es'
networks_endpoint = '/v2/networks'
# define helper function to make to avoid repeated code for subsequent calls
def get_city_bike_data(base_url,endpoint,params=None):
    url = city_bike_base_url + endpoint
    return requests.get(url,params=params)

In [79]:
response = get_city_bike_data(city_bike_base_url,networks_endpoint)

In [80]:
response.request.url

'http://api.citybik.es/v2/networks'

In [48]:
result = response.json()

In [51]:
df_networks= pd.json_normalize(result['networks'], sep='_') #sep used to format column names during flattening

In [52]:
df_networks.head()

,company,href,id,name,location_city,location_country,location_latitude,location_longitude,source,gbfs_href,license_name,license_url,ebikes
0,[ЗАО «СитиБайк»],/v2/networks/velobike-moscow,velobike-moscow,Velobike,Moscow,RU,55.750000,37.616667,NaN,NaN,NaN,NaN,NaN
1,[Urban Infrastructure Partner],/v2/networks/baerum-bysykkel,baerum-bysykkel,Bysykkel,Bærum,NO,59.894550,10.546343,NaN,NaN,NaN,NaN,NaN
2,[Comunicare S.r.l.],/v2/networks/bicincitta-siena,bicincitta-siena,Bicincittà,Siena,IT,43.318600,11.330600,https://www.bicincitta.com/frmLeStazioni.aspx?...,NaN,NaN,NaN,NaN
3,[Cyclopolis Systems],/v2/networks/cyclopolis-maroussi,cyclopolis-maroussi,Cyclopolis,Maroussi,GR,38.056872,23.808330,NaN,NaN,NaN,NaN,NaN
4,[Cyclopolis Systems],/v2/networks/cyclopolis-nafplio,cyclopolis-nafplio,Cyclopolis,Nafplio,GR,37.563940,22.809340,NaN,NaN,NaN,NaN,NaN


In [38]:
store_records = []
for network in result['networks']:
    city = network['location']['city']
    country = network['location']['country']
    network_id_endpoint = network['href']
    record = {
            'city': city,
            'country': country,
            'path': network_id_endpoint
        }
    store_records.append(record)

In [39]:
df_networks = pd.DataFrame(store_records)

In [41]:
df_networks.head()

,city,country,path
0,Moscow,RU,/v2/networks/velobike-moscow
1,Bærum,NO,/v2/networks/baerum-bysykkel
2,Siena,IT,/v2/networks/bicincitta-siena
3,Maroussi,GR,/v2/networks/cyclopolis-maroussi
4,Nafplio,GR,/v2/networks/cyclopolis-nafplio


In [54]:
df_networks['location_city'].value_counts().head(10)

location_city
Ljubljana      2
São Paulo      2
Bratislava     2
Igoumenitsa    2
Bergamo        2
Frankfurt      2
Köln           2
Hamburg        2
Berlin         2
Nicosia        2
Name: count, dtype: int64

In [ ]:
# I choose Berlin from the list of the cities with the most networks

In [55]:
df_networks.loc[df_networks['location_city']=='Berlin',:]

,company,href,id,name,location_city,location_country,location_latitude,location_longitude,source,gbfs_href,license_name,license_url,ebikes
344,[Nextbike GmbH],/v2/networks/nextbike-berlin,nextbike-berlin,Nextbike,Berlin,DE,52.5087,13.3563,NaN,NaN,NaN,NaN,NaN
619,None,/v2/networks/callabike-berlin,callabike-berlin,Call-A-Bike,Berlin,DE,52.5179,13.3895,NaN,https://apis.deutschebahn.com/db-api-marketpla...,NaN,NaN,NaN


In [60]:
Berlin_endpoints = df_networks.loc[df_networks['location_city']=='Berlin',:]['href'].to_list()

In [72]:
Berlin_endpoints

['/v2/networks/nextbike-berlin', '/v2/networks/callabike-berlin']

Send a request to CityBikes for the city of your choice. 

In [76]:
params ={'fields':'stations'}
store_networks = []
for endpoint in Berlin_endpoints:
    print(endpoint)
    response = get_city_bike_data(city_bike_base_url,endpoint,params=params)
    store_networks.append(response)

/v2/networks/nextbike-berlin
/v2/networks/callabike-berlin


In [82]:
for network in store_networks:
    print(network.request.url)

http://api.citybik.es/v2/networks/nextbike-berlin?fields=stations
http://api.citybik.es/v2/networks/callabike-berlin?fields=stations


Parse through the response to get the details you want for the bike stations in that city (latitude, longitude, number of bikes). 

In [120]:
berlin_1 = store_networks[0].json()

In [121]:
df_berlin1 = pd.json_normalize(berlin_1['network']['stations'],sep='_')

In [122]:
df_berlin1.head()

,empty_slots,free_bikes,id,latitude,longitude,name,timestamp,extra_bike_uids,extra_number,extra_slots,extra_uid
0,0,9,f5462555877f4cb1317baf6041527a93,52.504157,13.335328,virtuell - Kurfürstendamm/Rankestraße,2023-10-21T02:09:14.386000Z,"[19185, 18430, 18272, 17379, 14400, 14409, 198...",1640,4,44437
1,3,0,c32093893e4e32afee18894c7c0c1ba4,52.496986,13.291210,virtuell - EDEKA Schmitt (S Halensee),2023-10-21T02:09:14.388000Z,[],1670,4,72228
2,3,1,362ba47350cab100fe1d81be3cb7a058,52.498323,13.296157,Joachim-Friedrich-Str./KuDamm,2023-10-21T02:09:14.389000Z,[19954],1669,5,72229
3,2,2,07b6329ced662a2009a9132eef6ac711,52.499137,13.303743,Albrecht-Achilles-Straße,2023-10-21T02:09:14.390000Z,"[19525, 100778]",1672,5,72238
4,4,4,dac275f8454a83bf3e1e7ad1a4fb7e99,52.500614,13.294401,Karlsruher Str./Aspria,2023-10-21T02:09:14.391000Z,"[18889, 18823, 17491, 100665]",1671,8,72263


In [124]:
df_berlin1[['id','name','free_bikes','latitude','longitude']].head()

,id,name,free_bikes,latitude,longitude
0,f5462555877f4cb1317baf6041527a93,virtuell - Kurfürstendamm/Rankestraße,9,52.504157,13.335328
1,c32093893e4e32afee18894c7c0c1ba4,virtuell - EDEKA Schmitt (S Halensee),0,52.496986,13.291210
2,362ba47350cab100fe1d81be3cb7a058,Joachim-Friedrich-Str./KuDamm,1,52.498323,13.296157
3,07b6329ced662a2009a9132eef6ac711,Albrecht-Achilles-Straße,2,52.499137,13.303743
4,dac275f8454a83bf3e1e7ad1a4fb7e99,Karlsruher Str./Aspria,4,52.500614,13.294401


In [125]:
df_berlin1 = df_berlin1[['id','name','free_bikes','latitude','longitude']]

In [126]:
df_berlin1.head()

,id,name,free_bikes,latitude,longitude
0,f5462555877f4cb1317baf6041527a93,virtuell - Kurfürstendamm/Rankestraße,9,52.504157,13.335328
1,c32093893e4e32afee18894c7c0c1ba4,virtuell - EDEKA Schmitt (S Halensee),0,52.496986,13.291210
2,362ba47350cab100fe1d81be3cb7a058,Joachim-Friedrich-Str./KuDamm,1,52.498323,13.296157
3,07b6329ced662a2009a9132eef6ac711,Albrecht-Achilles-Straße,2,52.499137,13.303743
4,dac275f8454a83bf3e1e7ad1a4fb7e99,Karlsruher Str./Aspria,4,52.500614,13.294401


In [128]:
# repeat process for the second network in Berlin
berlin_2 = store_networks[1].json()
df_berlin2 = pd.json_normalize(berlin_2['network']['stations'],sep='_')
df_berlin2 = df_berlin2[['id','name','free_bikes','latitude','longitude']]

In [129]:
df_berlin2.head()

,id,name,free_bikes,latitude,longitude
0,bda776909eade3fa5ceebbe3bcc53126,Alt-Moabit / Gotzkowskystraße,1,52.523892,13.331252
1,f738c752a276f8a6867676741e1ef954,U Senefelder Platz / Schönhauser Allee,2,52.531871,13.412681
2,e29e347c096a6404c7f3d4d5d82ef451,Lützowplatz / Karl-Heinrich-Ulrichsstraße,3,52.503992,13.352739
3,be47c6f38184ff3996a2a8485f3ad4c0,Invalidenstraße / Lesser-Ury-Weg,2,52.524195,13.361548
4,9cec7be548cbb7c42203f3c51fc2cefd,Mühlenstraße / Mercedes Platz,4,52.504122,13.442253


Put your parsed results into a DataFrame.

In [130]:
df_berlin = pd.concat([df_berlin1,df_berlin2], axis=0)

In [131]:
df_berlin.head()

,id,name,free_bikes,latitude,longitude
0,f5462555877f4cb1317baf6041527a93,virtuell - Kurfürstendamm/Rankestraße,9,52.504157,13.335328
1,c32093893e4e32afee18894c7c0c1ba4,virtuell - EDEKA Schmitt (S Halensee),0,52.496986,13.291210
2,362ba47350cab100fe1d81be3cb7a058,Joachim-Friedrich-Str./KuDamm,1,52.498323,13.296157
3,07b6329ced662a2009a9132eef6ac711,Albrecht-Achilles-Straße,2,52.499137,13.303743
4,dac275f8454a83bf3e1e7ad1a4fb7e99,Karlsruher Str./Aspria,4,52.500614,13.294401


In [132]:
df_berlin.shape

(3890, 5)

In [133]:
df_berlin[df_berlin.duplicated()] #Ensure there are no duplicated entries

,id,name,free_bikes,latitude,longitude


In [134]:
# save data in a csv for future reuse
df_berlin.to_csv('city_bikes_data',index=False)